In [ ]:
import requests
import os

def check_URL(input_string):
    print("Welcome to IsItDown.py!")
    print("Please write a URL or URLs you want to check. (separated by comma)")
    li = input_string.replace(' ','').split(',')
    li = ['http://'+i.lower() if 'http://' not in i.lower() else i for i in li ]
    for site in li:
        try:
            if int(requests.get(site).status_code)//100 == 2: 
                print(f"{site} is up!")
            else:
                print(f"{site} is down!")
        except:
            print(f"{site} is down!")

def answer():
    repeat = input("Do you want to start over? y/n ")
    if repeat == 'n':
        return 0
    elif repeat == 'y':
        return 1
    else:
        print("Wrong Answer. again")
        return answer()
            
while True:
    input_string = input()
    check_URL(input_string)
    if answer():
        continue
    else:
        break    

## Asignment 6

In [ ]:
import os
import requests
from bs4 import BeautifulSoup

os.system("clear")

url = "https://www.iban.com/currency-codes"


countries = []

request = requests.get(url)
soup = BeautifulSoup(request.text, "html.parser")

table = soup.find("table")
rows = table.find_all("tr")[1:]

for row in rows:
    items = row.find_all("td")
    name = items[0].text
    code =items[2].text
    if name and code:
        if name != "No universal currency":
            country = {
                'name':name.capitalize(),
                'code': code
            }
            countries.append(country)


def ask():
    try:
        choice = int(input("#: "))
        if choice > len(countries):
            print("Choose a number from the list.")
            ask()
        else:
            country = countries[choice]
            print(f"{country['name']}\n")
    except ValueError:
        print("That wasn't a number.")
        ask()
    return country['code']

def ask_convert():
    print(f"\nHow many {country_a} do you want to convert to {country_b}?")
    money_input = input()
    try:
        money_input = float(money_input)
    except:
        print("That wasn't a number.")
        money_input = ask_convert()
    return money_input

def get_sym():
    symbol_url = "https://transferwise.com/gb/blog/world-currency-symbols"

    request = requests.get(symbol_url)
    soup = BeautifulSoup(request.text, 'html.parser')

    tables = soup.find_all("table")

    rows=[]
    for table in tables:
        rows.extend(table.find_all("tr")[1:])

    code_sym_dict=dict()

    for row in rows:
        items = row.find_all("td")
        code = items[2].text.strip()
        sym =items[3].text.strip()
        code_sym_dict[code] = sym
    return code_sym_dict
          
def convert(money,sym):
    rate_url = f"https://transferwise.com/gb/currency-converter/{country_a.lower()}-to-{country_b.lower()}-rate"
    request = requests.get(rate_url)
    soup = BeautifulSoup(request.text, 'html.parser')
    try:
        result = money * float(soup.select("h3 > span.text-success")[0].text)
    except:
          import pdb;pdb.set_trace()
    print(f"{country_a}{money:,.2f} is {sym}{result:,}")

                  
print("Hello! Please choose select a country by number:")
for index, country in enumerate(countries):
    print(f"#{index} {country['name']}")

print("Where are you from? Choose a country by number.\n")

country_a = ask()
print('Now choose another country.\n')
country_b = ask()
          
code_sym_dict = get_sym()

money = ask_convert()

try:
    sym = code_sym_dict[country_b]
except:
    sym = '$'

convert(money,sym)

## 나라랑 코드들을 받으면 유저가 2개의 나라를 선택하게 해.
## 유저가 나라a 에서 나라b로 변환하고 싶은 통화 량을 선택하게 해
## 2개의 화폐코드랑 양을 URL로 보내
##beautiful soup사용해서 Transfer Wise에서 변환 결과확인해서 가져와.

## Asignment 7

### http://www.alba.co.kr/ 의 
- ```html <ul class="goodBox">``` 에서 아래와같은 컬럼 추출
- place, title, time, pay, date



In [ ]:
import os
import csv
import requests
from bs4 import BeautifulSoup
import pandas as pd

def get_super_brand(url):
    request = requests.get(url)
    soup = BeautifulSoup(request.text, "html.parser")
    ## 슈퍼브랜드 li a태그 가져오기
    MainSuperBrand = soup.find('div',id='MainSuperBrand')
    ul = MainSuperBrand.find('ul',class_='goodsBox')
    superBrand_list = ul.find_all('li')

    superBrand_list = superBrand_list[:-1]

    href_list = []
    for superBrand in superBrand_list:
        company_name = superBrand.find('span',{'class':'company'}).text
        link = superBrand.a['href']
        href_list.append({'company':company_name,'link':link})
    return href_list

def get_table_row_li(super_brand_url):
    try:
        response = requests.get(super_brand_url['link'])
    except:
        import pdb;pdb.set_trace()

    super_brand_soup = BeautifulSoup(response.text, 'html.parser')

    ## place, title, time, pay, date
    NormalInfo = super_brand_soup.find('div', id='NormalInfo')
    table = NormalInfo.find('tbody')
    tr_li = table.find_all('tr')
    return super_brand_url['company'],tr_li
    

    
def get_job_info(table_row_li):
    brand_li=list()
    tr_li = table_row_li
    for tr in tr_li[::2]:
        try:
            place = tr.find('td',{'class':'local first'}).text.replace(u'\xa0', u' ')
        except:
            import pdb;pdb.set_trace()
        title = tr.find('td',{'class':'title'}).find('span',{'class':'company'}).text.strip()
        time = tr.find('td',{'class':'data'}).text
        pay = tr.find('td',{'class':'pay'}).text
        date = tr.find('td',{'class':'regDate last'}).text
        brand_li.append({'place':place, 'title':title, 'time':time,'pay':pay,'date':date})
    return brand_li

os.system("clear")
alba_url = "http://www.alba.co.kr"

href_list = get_super_brand(alba_url)
for one_company in href_list:
    company_name, table_row_li = get_table_row_li(one_company)
    if len(table_row_li) == 1:
        continue
    brand_li =  get_job_info(table_row_li)
    company_name = company_name.replace('/','_')
    pd.DataFrame(brand_li)[['place','title','time','pay','date']].to_csv(f'{company_name}.csv',index=False)
    print(f"{company_name} csv 생성완료!")

## Assignment 8-9

In [ ]:
import requests
from flask import Flask, render_template, request
import pandas as pd

base_url = "http://hn.algolia.com/api/v1"

# This URL gets the newest stories.
new = f"{base_url}/search_by_date?tags=story"

# This URL gets the most popular stories
popular = f"{base_url}/search?tags=story"

def extract_json(order_by):
    page={}
    r_page = requests.get(order_by)
    for r in r_page.json()['hits']:
        page.update({r['objectID'] : {'num_comments' : r['num_comments'],
                    'author' : r['author'],
                    'points' : r['points'],
                    'title' : r['title'],
                    'url' : r['url'],
                    'objectID' : r['objectID']
                    }})
    return page

# This function makes the URL to get the detail of a storie by id.
# Heres the documentation: https://hn.algolia.com/api
def make_detail_url(id):
    return f"{base_url}/items/{id}"

new_page_di = extract_json(new)
new_page_li = list(new_page_di.values())
popular_page_di = extract_json(popular)
popular_page_li = list(popular_page_di.values())

app = Flask("DayNine")

@app.route("/")
def home(**kwargs):
    order_by = request.args.get('order_by')
    if order_by:
        if order_by == 'new':
            return render_template('order_new.html',page_li=new_page_li)
        elif order_by == 'popular':
            
            return render_template('index.html',page_li=popular_page_li)
    else:
        return render_template('index.html')

@app.route("/<id>")
def go_id_page(id):
    if id in new_page_di.keys():
        news = new_page_di[str(id)]
    elif id in popular_page_di.keys():
        news = popular_page_di[str(id)]
    comment_url = make_detail_url(id)
    comment_page = requests.get(comment_url)
    df = pd.DataFrame(comment_page.json()['children'])
    df_comment = df[df.author.notna()][['author','text']]
    return render_template('detail.html', df=df_comment, news=news)

# @app.route("/?order_by=popular")
# def order_popular():
#     return render_template("index.html")

app.run(host="0.0.0.0")

## Last Assingment

In [ ]:
import requests
from flask import Flask, render_template, request
from bs4 import BeautifulSoup
import pandas as pd

"""
When you try to scrape reddit make sure to send the 'headers' on your request.
Reddit blocks scrappers so we have to include these headers to make reddit think
that we are a normal computer and not a python script.
How to use: requests.get(url, headers=headers)
"""

headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.47 Safari/537.36'}


"""
All subreddits have the same url:
i.e : https://reddit.com/r/javascript
You can add more subreddits to the list, just make sure they exist.
To make a request, use this url:
https://www.reddit.com/r/{subreddit}/top/?t=month
This will give you the top posts in per month.
"""

subreddits = [
    "javascript",
    "reactjs",
    "reactnative",
    "programming",
    "css",
    "golang",
    "flutter",
    "rust",
    "django"
]


app = Flask("DayEleven")

@app.route("/")
def home(**kwargs):
#     order_by = request.args.get('order_by')
    return render_template('home.html')

@app.route("/read")
def read(**kwargs):
    check_li=[]
    for sub in subreddits:
        check_li.append(request.args.get(sub))
        on_li = [subreddits[i] for i in [idx for idx,i in enumerate(check_li) if i=='on']]
    result_di={}
    for subreddit in on_li:
        url = f'https://www.reddit.com/r/{subreddit}/top/?t=month'
        r = requests.get(url, headers=headers)
        soup = BeautifulSoup(r.text, "html.parser")
        div_all = soup.find('div', class_='rpBJOHq2PR60pnwJlUyP0')
        div_li = div_all.find_all('div',class_='_1oQyIsiPHYt6nx7VOmd1sz')
        result_di[subreddit] = []
        for i in range(len(div_li)):
            try:
                vote_num_li = div_li[i].find_all('div',{'class':'_1rZYMD_4xY3gRcSS3p8ODO'})
                if len(vote_num_li) == 2:
                    if vote_num_li[0].text == vote_num_li[1].text:
                        vote_num = int(vote_num_li[0].text)
                else:
                    continue
                title = div_li[i].find('h3',class_='_eYtD2XCVieq6emjKBH3m').text
                link = 'https://www.reddit.com' + div_li[i].find('a',{'class':'SQnoC3ObvgnGjWt90zD9Z'})['href']
                result_di[subreddit].append([title,link,vote_num])
            except:
                pass

    df_main = pd.DataFrame(columns=['title','link','vote','kind'])

    for i in on_li:
        df = pd.DataFrame(result_di[i])
        df['kind'] = i
        df.columns=['title','link','vote','kind']
        df_main = pd.concat([df_main,df])

    df_main.sort_values('vote',ascending=0,inplace=True)
    return render_template('read.html',df=df_main,on_li=on_li)


app.run(host="0.0.0.0")

 * Serving Flask app "DayEleven" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [26/Dec/2020 03:50:50] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [26/Dec/2020 03:50:50] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [26/Dec/2020 03:51:05] "GET /read?golang=on&django=on HTTP/1.1" 200 -
